In [1]:
! pip install -r requirements.txt --quiet

ERROR: Could not find a version that satisfies the requirement azure-ai-agents--pre (from versions: none)
ERROR: No matching distribution found for azure-ai-agents--pre


# Creating and Deploying an Azure AI Agent Using SDK and MCP Tools

This notebook demonstrates how to create and deploy an Azure AI Agent using the Azure SDK, with tools generated from an OpenAPI endpoint. It provides a coded example that mirrors the portal example demonstrated in this workshop, offering a programmatic approach to deploying agents with OpenAPI-driven tools.


🔗 [How to use the Model Context Protocol tool](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples)

In [ ]:
from os import environ
import time
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam

load_dotenv()

load_dotenv(override=True)

True

## Get MCP server configuration from environment variables

In [61]:
mcp_server_url = environ["MCP_SERVER_URL"]
mcp_server_label = environ["MCP_SERVER_LABEL"] 

## Create a project client

In [ ]:
# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

openai_client = project_client.get_openai_client()

## Initialize agent MCP tool

In [ ]:

audit_mcp_tool = MCPTool(
        server_label=mcp_server_label,
        server_url=mcp_server_url,
        require_approval="never",  # set to "never" for smoother demos if desired

    )

print(f"Allowed tools: {audit_mcp_tool.allowed_tools}")

Allowed tools: []


## Create an agent and a thread

In [ ]:
instructions = """
You are an audit assistant. Use MCP tools to retrieve invoices, vendors, and payments from the audit data store. Do not invent records. When you make a claim, cite the specific invoice_id/vendor_id/payment_id you used.
"""

agent = project_client.agents.create_version(
    agent_name="AuditAgentPoC",
    definition=PromptAgentDefinition(
        model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
        instructions=instructions,
        tools=[audit_mcp_tool],
    ),
)
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

conversation = openai_client.conversations.create()
print(f"Created conversation (id: {conversation.id})")


Created agent, ID: asst_x52TCmXk1g9RDhCHXjUuVn8M
Created thread, ID: thread_scLbEM2fl8kPwLsubpQl0ZdT


In [ ]:
# Send a request that should trigger MCP tool calls
# (Your MCP server should expose tools like query_invoices/get_invoice/get_vendor/query_payments)
response = openai_client.responses.create(
    conversation=conversation.id,
    input=(
        "Engagement eng-001: Identify potential duplicate invoices in Q3 2025 "
        "(same vendor_id and same amount within 7 days), and list the invoice_ids found."
    ),
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)


Created message, ID: msg_OUzVe4T30QY4dh43ntw4dynw


In [ ]:
input_list: ResponseInputParam = []
for item in response.output:
    if item.type == "mcp_approval_request" and item.server_label == "audit-mcp" and item.id:
        input_list.append(
            McpApprovalResponse(
                type="mcp_approval_response",
                approve=True,
                approval_request_id=item.id,
            )
        )

# If approvals were needed, send them back to continue execution
if input_list:
    response = openai_client.responses.create(
        input=input_list,
        previous_response_id=response.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

print(f"Response: {response.output_text}")

Created run, ID: run_B2UpEHw7I2W6Z6qqp5xMw34i
Approving tool call: {'id': 'call_sPidgGU1u1szmFbz5hyywMD9', 'type': 'mcp', 'arguments': '{"exception_type":"Breakdown"}', 'name': 'get_exceptions_by_type', 'server_label': 'TransportOperations'}
tool_approvals: [{'tool_call_id': 'call_sPidgGU1u1szmFbz5hyywMD9', 'approve': True, 'headers': {}}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


In [ ]:
project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
print("Agent deleted")

Deleted agent
